In [ ]:
!pip install moviepy
!pip install ffmpeg-python

In [ ]:
import os
import re
from glob import glob

import moviepy.editor as mpy  # for title clip
import ffmpeg  # watermark, concatenate (w/ for fade-in, out), produce output

In [ ]:
print('Create temperary folder and output folder')
os.makedirs('../tmp',exist_ok=True)
os.makedirs('../output',exist_ok=True)
TMP_DIR = '/project/ta-shared-ii/video-handel-compress/tmp'
OUTPUT_DIR = '/project/ta-shared-ii/video-handel-compress/output'

In [ ]:
print('List all files from DATA_DIR with suffix in SUFFIX_LIST, find their title picture in ".png" format')

SUFFIX_LIST = ['.mkv', '.mp4', '.mov']
DATA_DIR = '/project/ta-shared-ii/video-handel-compress/data'
all_files = glob(DATA_DIR+'/*.*')
vfiles = []
for f in all_files:
    if re.findall(fr'{"|".join(SUFFIX_LIST)}',f):
        vfiles.append(f)
tfiles = list(
    map(
        lambda x: re.sub(f'{"|".join(SUFFIX_LIST)}', '.png', x),
        vfiles
    )
)
display(list(zip(vfiles,tfiles))[:3])

In [ ]:
## 因為FFMPEG做開頭段落影像會解碼失敗，所以使用moviepy，會慢點

print('Create title films')

# Prepare dummy audio file
make_frame = lambda t: 0.0
dummy_audio = mpy.AudioClip(make_frame, duration=3.5, fps=44100) 

tmp_filename = []
for f in tfiles:
    # title clip
    tmp_filename.append(f'{TMP_DIR}/{f.split("/")[-1][:-4]}_title.mkv')
    if not os.path.exists(tmp_filename[-1]):
        tclip = mpy.ImageClip(f, duration=3.5)
        tclip = tclip.set_audio(dummy_audio)
        tclip.write_videofile(tmp_filename[-1], codec='libx264', fps=30, bitrate='2500K')

In [ ]:
# 異質性video接合，並附上fade-in/ out

tfile = ffmpeg.input(tmp_filename[0])
vfile = ffmpeg.input('test.mkv')
joined = ffmpeg.concat(tfile.video.filter_('fade', t='out', st=3.0, d=0.5),
                       tfile.audio,
                       vfile.video.filter_('fade', t='in', st=0, d=0.5),
                       vfile.audio,
                       v=1, a=1)
out = ffmpeg.output(joined, f'{OUTPUT_DIR}/out.mp4',r=30)
out.run()

In [ ]:
# 加上watermark
vfile = ffmpeg.input('test.mkv')
wmfile = ffmpeg.input(DATA_DIR+'/watermark.png')

fused = ffmpeg.overlay(vfile.video, wmfile)
out = ffmpeg.output(fused, vfile.audio, f'{OUTPUT_DIR}/out2.mp4',r=30)
out.run()

In [ ]:
# 先上watermark, 再做異質性video接合，並附上fade-in/ out
tfile = ffmpeg.input(tmp_filename[0])
vfile = ffmpeg.input('test.mkv')
wmfile = ffmpeg.input(DATA_DIR+'/watermark.png')


fused = ffmpeg.overlay(vfile.video, wmfile)

joined = ffmpeg.concat(tfile.video.filter_('fade', t='out', st=3.0, d=0.5),
                       tfile.audio,
                       fused.filter_('fade', t='in', st=0, d=0.5),
                       vfile.audio,
                       v=1, a=1)
out = ffmpeg.output(joined, f'{OUTPUT_DIR}/do_all.mp4',r=30)
out.run()

### Template:
- https://github.com/kkroening/ffmpeg-python/blob/master/examples/README.md
- https://dev.to/dak425/add-fade-in-and-fade-out-effects-with-ffmpeg-2bj7